In [1]:
import nanohubremote as nr
import pandas as pd
import pandas as pd
import plotly.graph_objs as go
pd.options.plotting.backend = "plotly"
import os
import numpy as np

In [2]:
auth_data = {
    'grant_type' : 'tool',
}
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"] 

In [15]:
import json
session = nr.Session(auth_data)
search = {
    'tool':'cellrelaxdft', 
    'filters':json.dumps([
        {'field':'input.mp_id','operation':'=','value':'mp-1009127'},
        {'field':'output.energy','operation':'>','value':-2492.0},
        {'field':'output.pressure','operation':'>','value':500},
        {'field':'output.pressure','operation':'<','value':700},
    ]),
    'results':json.dumps([
        'input.k_x',
        'input.k_y',
        'input.k_z',
        'output.pressure',
        'output.energy',
        'output.lattice_vectors',
        'output.lattice_parameters',
    ]),
    'simtool' : True,
    'revision' : 49,
}
req_json = session.requestPost('results/dbexplorer/search', data=search)
req_json = req_json.json()
data = pd.DataFrame([p for p in req_json["results"]])


In [16]:
data

,squid,revision,date,input.k_x,input.k_y,input.k_z,output.pressure,output.energy,output.lattice_vectors,output.lattice_parameters
0,cellrelaxdft/r49/47ff7e066a6401a6477536e4aa0ce...,r49,2023-01-30 10:30:20.328198,11,11,11,500.508,-2484.976052,"[[2.0685699352255, 0, 0], [0, 2.0685699352255,...","[2.0685699352255, 2.0685699352255, 2.068569935..."
1,cellrelaxdft/r49/1d518f4309a54618d9b5b9a6bf310...,r49,2023-01-30 10:30:20.328198,15,15,15,500.679,-2485.046653,"[[2.0691295834947, 0, 0], [0, 2.0691295834947,...","[2.0691295834947, 2.0691295834947, 2.069129583..."
2,cellrelaxdft/r49/3c3e1802e3837b06022a8fa963ca4...,r49,2023-01-30 10:30:20.328198,24,24,24,500.525,-2484.975806,"[[2.0685625221734, 0, 0], [0, 2.0685625221734,...","[2.0685625221734, 2.0685625221734, 2.068562522..."
3,cellrelaxdft/r49/70518ce414457eceb7de0e60eb9a6...,r49,2023-01-30 10:30:20.328198,24,24,24,500.549,-2485.048753,"[[2.0691818868132, 0, 0], [0, 2.0691818868132,...","[2.0691818868132, 2.0691818868132, 2.069181886..."
4,cellrelaxdft/r49/7f4deef34626a039f70eb6aea7e3e...,r49,2023-01-30 10:30:20.328198,15,15,15,500.520,-2484.975873,"[[2.0685646010557, 0, 0], [0, 2.0685646010557,...","[2.0685646010557, 2.0685646010557, 2.068564601..."
5,cellrelaxdft/r49/af584228b2df7c0e6b02e84eaee06...,r49,2023-01-30 10:30:20.328198,15,15,15,500.781,-2485.029031,"[[2.0689084086844, 0, 0], [0, 2.0689084086844,...","[2.0689084086844, 2.0689084086844, 2.068908408..."
6,cellrelaxdft/r49/b8602ce9387559a59a10f0d743d58...,r49,2023-01-30 10:30:20.328198,24,24,24,500.806,-2485.028634,"[[2.0688984984255, 0, 0], [0, 2.0688984984255,...","[2.0688984984255, 2.0688984984255, 2.068898498..."
7,cellrelaxdft/r49/976b6f7d04ec9a2a7e2adf984fb5a...,r49,2023-01-30 10:30:20.328198,11,11,11,500.609,-2485.031806,"[[2.0689777167927, 0, 0], [0, 2.0689777167927,...","[2.0689777167927, 2.0689777167927, 2.068977716..."
8,cellrelaxdft/r49/bbd279e8a05e633c29a9fe5d94541...,r49,2023-01-30 10:30:20.328198,11,11,11,500.653,-2485.047080,"[[2.0691404217123, 0, 0], [0, 2.0691404217123,...","[2.0691404217123, 2.0691404217123, 2.069140421..."
9,cellrelaxdft/r49/20642ac50d91bd8e5f8b18ca5e7c9...,r49,2023-01-30 10:30:20.328198,8,8,8,500.546,-2484.975467,"[[2.0685581966544, 0, 0], [0, 2.0685581966544,...","[2.0685581966544, 2.0685581966544, 2.068558196..."


In [18]:
data.plot.scatter(x="output.pressure", y="output.energy")

In [5]:
search = {
    'tool':'pntoy', 
    'filters':json.dumps([        
        {'field':'input.p_len','value':4,'operation':'=='},
        {'field':'input.n_len','value':4,'operation':'=='},
        {'field':'input.temperature','value':300,'operation':'=='},
        {'field':'input.i_len','value':0,'operation':'=='},
        {'field':'input.materialp','value':"Si",'operation':'=='},
        {'field':'input.impurity','value':'false','operation':'=='},
    ]),
    'results':json.dumps([
        'input.Na', 
        'input.Nd',
        'output.p',
    ]),    
    'limit':10,    
}
req_json = session.requestPost('results/dbexplorer/search', data=search)
req_json = req_json.json()
df = pd.DataFrame(req_json['results'])


In [7]:
data = df.apply(lambda a : {"x":a['output.p']['xaxis'],"y":a['output.p']['yaxis'], 'name':a['squid'],'type':'scatter'}, axis=1).tolist()

In [13]:
go.FigureWidget(data = data, layout={'yaxis':{'type':'log', 'exponentformat':"e"}})

FigureWidget({
    'data': [{'name': 'pntoy/56/11713d7fa63c460f57ed06ccec4a39c92511d234',
              'type'…